In [5]:
from google.colab import drive
from os.path import join 

ROOT = "/content/gdrive/"
drive.mount(ROOT)
%cd /content/gdrive/Shareddrives/machine_learning_in_practice/Analog-PILGI-to-DIgital/object_detection

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
/content/gdrive/Shareddrives/machine_learning_in_practice/Analog-PILGI-to-DIgital/object_detection


In [7]:
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '3'
import torch
from torch.utils.data import DataLoader
from torch import optim
import numpy as np
import argparse

import config
from ctpn_model import CTPN_Model, RPN_CLS_Loss, RPN_REGR_Loss
from dataset import ICDARDataset


random_seed = 2019
torch.random.manual_seed(random_seed)
np.random.seed(random_seed)

epochs = 2
lr = 1e-3
resume_epoch = 0


def save_checkpoint(state, epoch, loss_cls, loss_regr, loss, ext='pth'):
    check_path = os.path.join(config.checkpoints_dir,
                              f'v3_ctpn_ep{epoch:02d}_'
                              f'{loss_cls:.4f}_{loss_regr:.4f}_{loss:.4f}.{ext}')

    try:
        torch.save(state, check_path)
    except BaseException as e:
        print(e)
        print('fail to save to {}'.format(check_path))
    print('saving to {}'.format(check_path))

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)


if __name__ == '__main__':
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    checkpoints_weight = config.pretrained_weights
    print('exist pretrained ',os.path.exists(checkpoints_weight))
    if os.path.exists(checkpoints_weight):
        pretrained = False

    dataset = ICDARDataset(config.icdar17_mlt_img_dir, config.icdar17_mlt_gt_dir)
    dataloader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=config.num_workers)
    model = CTPN_Model()
    model.to(device)
    
    if os.path.exists(checkpoints_weight):
        print('using pretrained weight: {}'.format(checkpoints_weight))
        cc = torch.load(checkpoints_weight, map_location=device)
        model.load_state_dict(cc['model_state_dict'])
        resume_epoch = cc['epoch']
    else:
        model.apply(weights_init)

    params_to_uodate = model.parameters()
    optimizer = optim.Adam(params_to_uodate, lr=lr)
    
    critetion_cls = RPN_CLS_Loss(device)
    critetion_regr = RPN_REGR_Loss(device)
    
    best_loss_cls = 100
    best_loss_regr = 100
    best_loss = 100
    best_model = None
    epochs += resume_epoch
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    
    for epoch in range(resume_epoch+1, epochs):
        print(f'Epoch {epoch}/{epochs}')
        print('#'*50)
        epoch_size = len(dataset) // 1
        model.train()
        epoch_loss_cls = 0
        epoch_loss_regr = 0
        epoch_loss = 0
        scheduler.step(epoch)
    
        for batch_i, (imgs, clss, regrs) in enumerate(dataloader):
            # print(imgs.shape)
            imgs = imgs.to(device)
            clss = clss.to(device)
            regrs = regrs.to(device)
    
            optimizer.zero_grad()
    
            out_cls, out_regr = model(imgs)
            loss_cls = critetion_cls(out_cls, clss)
            loss_regr = critetion_regr(out_regr, regrs)
    
            loss = loss_cls + loss_regr  # total loss
            loss.backward()
            optimizer.step()
    
            epoch_loss_cls += loss_cls.item()
            epoch_loss_regr += loss_regr.item()
            epoch_loss += loss.item()
            mmp = batch_i+1
    
            print(f'Ep:{epoch}/{epochs-1}--'
                  f'Batch:{batch_i}/{epoch_size}\n'
                  f'batch: loss_cls:{loss_cls.item():.4f}--loss_regr:{loss_regr.item():.4f}--loss:{loss.item():.4f}\n'
                  f'Epoch: loss_cls:{epoch_loss_cls/mmp:.4f}--loss_regr:{epoch_loss_regr/mmp:.4f}--'
                  f'loss:{epoch_loss/mmp:.4f}\n')
    
        epoch_loss_cls /= epoch_size
        epoch_loss_regr /= epoch_size
        epoch_loss /= epoch_size
        print(f'Epoch:{epoch}--{epoch_loss_cls:.4f}--{epoch_loss_regr:.4f}--{epoch_loss:.4f}')
        if best_loss_cls > epoch_loss_cls or best_loss_regr > epoch_loss_regr or best_loss > epoch_loss:
            best_loss = epoch_loss
            best_loss_regr = epoch_loss_regr
            best_loss_cls = epoch_loss_cls
            best_model = model
            save_checkpoint({'model_state_dict': best_model.state_dict(),
                             'epoch': epoch},
                            epoch,
                            best_loss_cls,
                            best_loss_regr,
                            best_loss)
    
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

exist pretrained  False
Epoch 1/2
##################################################


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:154: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://git

Ep:1/1--Batch:0/1000
batch: loss_cls:0.7200--loss_regr:0.1638--loss:0.8837
Epoch: loss_cls:0.7200--loss_regr:0.1638--loss:0.8837

Ep:1/1--Batch:1/1000
batch: loss_cls:0.8900--loss_regr:0.2499--loss:1.1399
Epoch: loss_cls:0.8050--loss_regr:0.2069--loss:1.0118

Ep:1/1--Batch:2/1000
batch: loss_cls:0.7827--loss_regr:0.2063--loss:0.9890
Epoch: loss_cls:0.7976--loss_regr:0.2067--loss:1.0042

Ep:1/1--Batch:3/1000
batch: loss_cls:0.7918--loss_regr:0.3289--loss:1.1207
Epoch: loss_cls:0.7961--loss_regr:0.2372--loss:1.0333

Ep:1/1--Batch:4/1000
batch: loss_cls:0.7551--loss_regr:0.1461--loss:0.9011
Epoch: loss_cls:0.7879--loss_regr:0.2190--loss:1.0069

Ep:1/1--Batch:5/1000
batch: loss_cls:0.7466--loss_regr:0.2229--loss:0.9695
Epoch: loss_cls:0.7810--loss_regr:0.2196--loss:1.0007

Ep:1/1--Batch:6/1000
batch: loss_cls:0.7375--loss_regr:0.2758--loss:1.0133
Epoch: loss_cls:0.7748--loss_regr:0.2277--loss:1.0025

Ep:1/1--Batch:7/1000
batch: loss_cls:0.7186--loss_regr:0.0938--loss:0.8124
Epoch: loss_cls

In [ ]:
! pwd
! git status

/content/gdrive/Shareddrives/machine_learning_in_practice/Analog-PILGI-to-DIgital/object_detection
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   ../.gitignore
	modified:   ../GAN/Parsing.ipynb
	modified:   ../GAN/WGAN_DIV.ipynb
	modified:   "github_util.ipynb\341\204\213\341\205\264 \341\204\211\341\205\241\341\204\207\341\205\251\341\206\253\341\204\213\341\205\264 \341\204\211\341\205\241\341\204\207\341\205\251\341\206\253"

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	../GAN/outputs/
	__pycache__/
	config.py
	ctpn_model.py
	ctpn_utils.py
	dataset.py
	error_imgs.txt
	test_data_generating_file/data_crawer.ipynb
	test_data_generating_file/index.html?com=downloads
	text_detector.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
enumerate(dataloader)
#checkpoints_weight = config.pretrained_weights


test_data_generating_file/index.html?com=downloads
test_data_generating_file/data_crawer.ipynb
test_data_generating_file/1.hwp


In [ ]:
config.pretrained_weights

'checkpoints/v3_ctpn_ep22_0.3801_0.0971_0.4773.pth'